## 2.	Напишите анимацию, в которой есть мяч, которым можно управлять с помощью клавиш на клавиатуре. Также добавьте случайное появление мишеней на экране, которые нужно поймать мячом. Сделать подсчет очков и отображать на экране количество очков, для отображения изменений по сумме очков использовать метод itemconfigure(). Очки начисляются за каждую пойманную мишень и уменьшаются за пропущенные мишени.

In [1]:
from tkinter import *
import random

# Функция для движения мяча влево при нажатии клавиши
def move_left(event):
    ball_x, ball_y, ball_x2, ball_y2 = canvas.coords(ball)
    if ball_x > 0:  # находится ли мяч уже у левого края
        canvas.move(ball, -10, 0)

# Функция для движения мяча вправо при нажатии клавиши
def move_right(event):
    ball_x, ball_y, ball_x2, ball_y2 = canvas.coords(ball)
    if ball_x2 < canvas_width:  # находится ли мяч уже у правого края
        canvas.move(ball, 10, 0)

# Функция для начала игры, вызывается при нажатии кнопки "Start"
def start_game():
    start_button.destroy()  # Удаляем кнопку "Start"
    update()  # Запускаем основной цикл игры

# Основная функция для обновления игрового состояния
def update():
    for square in squares:
        canvas.move(square, 0, square_speed)  # Перемещаем каждый квадрат вниз на значение `square_speed`
        square_coords = canvas.coords(square)  # Получаем координаты текущего квадрата
        if square_coords[1] > canvas_height:
            remove_square(square)  # Удаляем квадрат, если он достиг нижней границы
            missed_square()  # Обрабатываем случай, когда игрок пропустил квадрат
    
    for line in lines:
        canvas.move(line, 0, line_speed)  # Перемещаем каждую линию вниз на значение `line_speed`
        line_coords = canvas.coords(line)  # Получаем координаты текущей линии
        if line_coords[1] > canvas_height:
            line_y = -line_height  # Сбрасываем линию наверх
            canvas.coords(line, canvas_width // 2 - line_width // 2, 
                          line_y, canvas_width // 2 + line_width // 2, line_y + line_height)
    
    ball_coords = canvas.coords(ball)  # Получаем координаты мяча
    for square in squares:
        square_coords = canvas.coords(square)  # Получаем координаты текущего квадрата
        if ball_coords[2] >= square_coords[0] and ball_coords[0] <= square_coords[2] and ball_coords[3] >= square_coords[1] and ball_coords[1] <= square_coords[3]:
            collect_square(square)  # Обрабатываем случай, когда мяч собирает квадрат

    if len(squares) < max_squares and random.randint(0, 100) < square_spawn_chance:
        create_square()  # Создаем новый квадрат, если количество квадратов меньше максимального и выполняется условие для появления

    root.after(update_interval, update)  # Запускаем функцию обновления через определенный интервал времени

# Функция для создания нового квадрата
def create_square():
    square_width = 20
    square_height = 20
    square_x = random.randint(50, canvas_width - square_width - 50)
    square_color = random.choice(["#22eb0c", "red", "yellow"])
    square = canvas.create_rectangle(square_x, -square_height, square_x + square_width, 0, fill=square_color)
    squares.append(square)

# Функция для обработки случая, когда игрок пропускает квадратик
def missed_square():
    global score
    score -= 1
    update_score(0)

# Функция для удаления квадрата
def remove_square(square):
    canvas.delete(square)
    squares.remove(square)

# Функция для сбора квадрата и увеличения счета
def collect_square(square):
    remove_square(square)
    update_score(1)

# Фунеция для обновления счета в игре
def update_score(change):
    global score
    score += change
    score_label.config(text="Score: " + str(score))

# Создаем основное окно приложения
root = Tk()
root.title("Catch the Squares")  # Задаем заголовок окна

canvas_width = 600
canvas_height = 650
canvas = Canvas(root, width=canvas_width, height=canvas_height, bg="#444447")  # Создаем холст для отображения игры
canvas.pack()  # Размещаем холст в окне

line_width = 10
line_height = 40
line_spacing = 60
num_lines = canvas_height // line_spacing
lines = []

# Создаем начальные линии на холсте
for i in range(num_lines):
    line_y = i * line_spacing
    line = canvas.create_rectangle(canvas_width // 2 - line_width // 2, line_y, canvas_width // 2 + line_width // 2, line_y + line_height, fill="white")
    lines.append(line)

squares = []

max_squares = 4  # Максимальное количество квадратов на экране
square_spawn_chance = 2  # Шанс появления нового квадрата
square_speed = 3  # Скорость движения квадратов
line_speed = 3  # Скорость движения линий

update_interval = 10  # Интервал обновления игры (миллисекунды)

score = 0
score_label = Label(root, text="Score: " + str(score), font=("Arial", 16))  # Создаем метку для отображения счета
score_label.pack()
score_label.place(x=10, y=10)

ball_diameter = 60
ball_x = (canvas_width - ball_diameter) // 2
ball_y = canvas_height - ball_diameter - 50  # Начальные координаты мяча
ball = canvas.create_oval(ball_x, ball_y, ball_x + ball_diameter, ball_y + ball_diameter, fill="#0ccaeb")  # Создаем мяч

canvas.focus_set()
canvas.bind("<Left>", move_left)
canvas.bind("<Right>", move_right)

# Создаем кнопку "Start" и размещаем ее по середине
start_button = Button(root, text="Start", command=start_game, width=15, height=2)  # Устанавливаем ширину и высоту кнопки
start_button.pack()
start_button.place(relx=0.5, rely=0.5, anchor=CENTER)  # Размещаем кнопку по середине
start_button.configure(font=("Arial", 20))  # Устанавливаем размер шрифта кнопки

root.mainloop()  # Запускаем цикл обработки событий Tkinter
